### 0.02. Text with moving letters

In [ ]:
! pip install scipy

In [10]:
import numpy as np

from moviepy.editor import *
from moviepy.video.tools.segmenting import findObjects

# WE CREATE THE TEXT THAT IS GOING TO MOVE, WE CENTER IT.

screensize = (720,460)
txtClip = TextClip('Cool effect',color='white', font="Amiri-Bold",
                   kerning = 5, fontsize=100)
cvc = CompositeVideoClip( [txtClip.set_pos('center')],
                        size=screensize)

# THE NEXT FOUR FUNCTIONS DEFINE FOUR WAYS OF MOVING THE LETTERS


# helper function
rotMatrix = lambda a: np.array( [[np.cos(a),np.sin(a)], 
                                 [-np.sin(a),np.cos(a)]] )

def vortex(screenpos,i,nletters):
    d = lambda t : 1.0/(0.3+t**8) #damping
    a = i*np.pi/ nletters # angle of the movement
    v = rotMatrix(a).dot([-1,0])
    if i%2 : v[1] = -v[1]
    return lambda t: screenpos+400*d(t)*rotMatrix(0.5*d(t)*a).dot(v)
    
def cascade(screenpos,i,nletters):
    v = np.array([0,-1])
    d = lambda t : 1 if t<0 else abs(np.sinc(t)/(1+t**4))
    return lambda t: screenpos+v*400*d(t-0.15*i)

def arrive(screenpos,i,nletters):
    v = np.array([-1,0])
    d = lambda t : max(0, 3-3*t)
    return lambda t: screenpos-400*v*d(t-0.2*i)
    
def vortexout(screenpos,i,nletters):
    d = lambda t : max(0,t) #damping
    a = i*np.pi/ nletters # angle of the movement
    v = rotMatrix(a).dot([-1,0])
    if i%2 : v[1] = -v[1]
    return lambda t: screenpos+400*d(t-0.1*i)*rotMatrix(-0.2*d(t)*a).dot(v)


# WE USE THE PLUGIN findObjects TO LOCATE AND SEPARATE EACH LETTER

letters = findObjects(cvc) # a list of ImageClips


# WE ANIMATE THE LETTERS

def moveLetters(letters, funcpos):
    return [ letter.set_pos(funcpos(letter.screenpos,i,len(letters)))
              for i,letter in enumerate(letters)]

clips = [ CompositeVideoClip( moveLetters(letters,funcpos),
                              size = screensize).subclip(0,5)
          for funcpos in [vortex, cascade, arrive, vortexout] ]

# WE CONCATENATE EVERYTHING AND WRITE TO A FILE

final_clip = concatenate_videoclips(clips)
# final_clip.write_videofile('./coolTextEffects.avi',fps=25,codec='mpeg4')
# final_clip.write_videofile('./coolTextEffects.mov',fps=25,codec='mpeg4')
final_clip.write_videofile('./coolTextEffects.mp4', fps=25)

Moviepy - Building video ./coolTextEffects.mp4.
Moviepy - Writing video ./coolTextEffects.mp4



Moviepy - Done !
Moviepy - video ready ./coolTextEffects.mp4


In [11]:
# ! OFILE='coolTextEffects.mp4' && touch $OFILE && rm -f $OFILE && ! ffmpeg -loglevel quiet -i coolTextEffects.mov $OFILE
# ! ffmpeg -loglevel quiet -i coolTextEffects.mov coolTextEffects.mp4
# ! ffmpeg -loglevel quiet -i coolTextEffects.mov -c:v libx264 -c:a aac -strict experimental -b:a 192k -shortest coolTextEffects.mp4

! ls -l cool*

-rw-r--r--  1 kang-air  staff  803484  4  1 15:57 coolTextEffects.mov
-rw-r--r--  1 kang-air  staff   78944  4  1 16:19 coolTextEffects.mp4
